In [1]:
import time
import datetime
import re
from collections import OrderedDict
from selenium import webdriver
from selenium.webdriver.common.action_chains import ActionChains
from selenium.common.exceptions import NoSuchElementException
from selenium.common.exceptions import WebDriverException
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import pymongo
from pymongo import MongoClient
from pprint import pprint

In [2]:
def writeLog(log_text, function_name):
    with open('web_parcer.log', 'a') as file:
        file.write(str(datetime.datetime.now()) + ":\t" + function_name + ': \t' + log_text + '\n')


def goToPage(query, driver):
    start_loading = datetime.datetime.now()
    try:
        driver.set_page_load_timeout(120)
        driver.get("https://www.aviasales.ru/search/{}".format(query))
    except WebDriverException:
        writeLog('Page loading timeout :' + query, 'goToPage')
    
    try:
        time.sleep(3)    
        wait_seconds = 600
        wait = WebDriverWait(driver, wait_seconds)
        wait.until(EC.invisibility_of_element((By.CLASS_NAME, "countdown")))
        loading_duration = (datetime.datetime.now() - start_loading).seconds
        printDelimiter('*')
        print("$$$$$   page loaded for {} seconds   $$$$$".format(loading_duration))
        time.sleep(1)
    except:
        print("\x1b[31m\"CAN'T WAIT THE PAGE LOAD ({} seconds)\"\x1b[0m".format(wait_seconds))
    

def closeExtraTab(driver): # В предположении что драйвер находится на рекламном окне после его открытия
    num_of_tabs = len(driver.window_handles)
    if num_of_tabs == 2:
        main_window_handle = None
        if driver.current_window_handle == driver.window_handles[0]:
            driver.switch_to_window(driver.window_handles[1])
            main_window_handle = driver.window_handles[0]
        elif driver.current_window_handle == driver.window_handles[1]:
            driver.switch_to_window(driver.window_handles[0])
            main_window_handle = driver.window_handles[1]
        title_of_closed_tab = driver.title
        driver.close()
        driver.switch_to_window(main_window_handle)
        writeLog("Extra tab *{}* has been closed".format(title_of_closed_tab), 'closeExtraTab')
    
    
def printDelimiter(char):
    delimiter = ''
    for i in range(118):
        delimiter += char
    print(delimiter)
    
    
def getCollection():
    client = MongoClient()
    db = client.aviasales_database # getting a database
    return db.ticket_collection

In [3]:
#hardcoded_year = None################

def processPage(driver, collection):
    tickets = driver.find_elements_by_class_name("ticket__wrapper")
    tickets = removeAdvertisingTickets(tickets)
    tickets = removeAbsentTickets(tickets)
    #print(tickets)
    print(driver.title[-9:], '\n')
    for ticket in tickets:
        collection.insert_one(processTicket(ticket))
        closeExtraTab(driver)
        
        
def removeAdvertisingTickets(tickets):
    for ticket in tickets:
        ticket_opener_text = ticket.find_element_by_class_name("ticket__opener").text
        if ticket_opener_text == 'Реклама':
            tickets.remove(ticket)
    return tickets


def removeAbsentTickets(tickets):
    for ticket in tickets:
        ticket_price = (ticket.find_element_by_class_name("buy-button__price").text).replace('\u2009', '')
        if ticket_price == '':
            tickets.remove(ticket)
    return tickets
        
        
def processTicket(ticket):
    ticket_price = int((ticket.find_element_by_class_name("buy-button__price-num").text).replace('\u2009', ''))
    
    ticket_opener = ticket.find_element_by_class_name("ticket__opener")
    ticket_opener.click()
    
    ticket_content = ticket.find_element_by_class_name("ticket__content")
    
    flights = ticket_content.find_elements_by_class_name("ticket__segment.segment")
    
    print("crawl_date_time: ", datetime.datetime.now())                    # debug
    print("price: ", ticket_price)                                         # debug
    
    post = dict()
    post['crawl_date_time'] = datetime.datetime.now()
    post['price'] = ticket_price
    post['flights'] = list()
    
    for flight in flights:
        post['flights'].append(processFlight(post, flight))
    
    return post

        
def processFlight(post, flight):
    flight_info = dict()
    flight_info['details'] = list()
    
    general_flight_info = getGeneralFlightInfo(flight)
    flight_info['city'] = general_flight_info[0]
    flight_info['date_time'] = general_flight_info[1]
    flight_info['total_time'] = general_flight_info[2]
    
    #nonlocal hardcoded_year
    #hardcoded_year = flight_general_info['date_time']['from'].year#############################
    
    
    detailed_flight_info = getDetailedFlightInfo(flight)
    for info in detailed_flight_info:
        flight_info['details'].append(info)
        
    flight_info_order = ['total_time', 'date_time', 'city']        # debug
    for info in flight_info_order:                                 # debug
        print('\t' + info + ": ", flight_info[info])               # debug
        
    print('\n')

    return flight_info
    
    
def getGeneralFlightInfo(flight):
# Общая информация
    flight_from = getGeneralPointInfo(flight, "segment-route__origin")

    flight_info_path = getGeneralPathInfo(flight)
    
    flight_to = getGeneralPointInfo(flight, "segment-route__destination")
    
    flight_info_city = {'from' : flight_from[0],
                        'to' : flight_to[0]}
    flight_info_date_time = {'from' : flight_from[1],
                             'to' : flight_to[1]}
    
    return flight_info_city, flight_info_date_time, flight_info_path


def getGeneralPathInfo(flight):
    flight_path = flight.find_element_by_class_name("segment-route__path")
    flight_total_time = re.split(' ', flight_path.find_element_by_class_name("segment-route__total-time").text)
    flight_total_time_info = getHoursAndMinutes(flight_total_time)
    return flight_total_time_info
    
    
def getHoursAndMinutes(time_string):
    hours = 0
    minutes = 0
    for word in time_string:
        if word[-1] == 'ч':
            hours = word[:-1]
        elif word[-1] == 'м':
            minutes = word[:-1]
    return {'hours' : hours, 'minutes' : minutes}
    
    
def getGeneralPointInfo(flight, segment_route):
    flight_point = flight.find_element_by_class_name(segment_route)
    flight_city = flight_point.find_element_by_class_name("segment-route__city").text
    time_point = re.split(':', flight_point.find_element_by_class_name("segment-route__time").text)
    date_point = re.split(' ', flight_point.find_element_by_class_name("segment-route__date").text)
    flight_date_time_point = datetime.datetime(int(date_point[2][:-1]), 
                                               translateDate(date_point[1]), 
                                               int(date_point[0]), 
                                               int(time_point[0]), 
                                               int(time_point[1]))
    return flight_city, flight_date_time_point

    
def translateDate(month):
    date_translator = {'янв' : 1, 'фев' : 2, 'мар' : 3, 'апр' : 4 ,'май' : 5 ,'июн' : 6, 'июл' : 7, 'авг' : 8,
                       'сен' : 9, 'окт' : 10, 'ноя' : 11, 'дек' : 12}
    return date_translator[month]
    
    
def getDetailedFlightInfo(flight):
    flight_info = list()
    try:
    # Детали перелета
        time.sleep(1)
    
        flight_itinerary = flight.find_element_by_class_name("segment__flights")

        # Flight legs
        flight_legs = flight_itinerary.find_elements_by_class_name("flight-leg")
        flight_legs_info = list()

        for flight_leg in flight_legs:
            flight_legs_info.append(getDetailedFlightLegInfo(flight_leg))


        # Flight stops    
        flight_stops = flight_itinerary.find_elements_by_class_name("flight-stop")
        flight_stops_info = list()

        for flight_stop in flight_stops:
            flight_stops_info.append(getDetailedFlightStopInfo(flight_stop))


        # Миксую через один инфу по остановкам и инфу по пересадкам (для восстановления хронологии в выводе)
        flight_info = mixFlightInfo(flight_legs_info, flight_stops_info)

        #for info in flight_info:                                               # debug
        #    if info['type'] == 'leg':                                          # debug
        #        print("\n\tflight date time: ", info['date_time'],             # debug
        #              "\n\t\tairport: ", info['airport'],                      # debug
        #              "\n\t\tairline: ", info['airline'],                      # debug
        #              "\n\t\tflight number: ", info['flight_number'])          # debug
        #    elif info['type'] == 'stop':                                       # debug
        #        print("\n\tstop time: ", info['time'],                         # debug
        #              "\n\tstop place: ", info['place'])                       # debug

        #print("\n")                                                        # debug
    except NoSuchElementException as exception:
        #print("\x1b[31m\"red\"\x1b[0m")
        print("\x1b[31m\"Element not found (getDetailedFlightInfo): ", exception, "\"\x1b[0m")
    return flight_info 
    
                                 
def mixFlightInfo(flight_legs_info, flight_stops_info):
    flight_info = [flight_stops_info[i//2] 
                   if i%2 else flight_legs_info[i//2] 
                   for i in range(len(flight_stops_info)*2+1)]
    return flight_info
                                 
    
def getDetailedFlightStopInfo(flight_stop):
    flight_attrs_tmp = dict()
    flight_attrs_tmp['place'] = flight_stop.find_element_by_class_name("flight-stop__stop-place").text
    flight_stop_time = re.split(' ', flight_stop.find_element_by_class_name("flight-stop__stop-time").text)
    flight_attrs_tmp['time'] = getHoursAndMinutes(flight_stop_time)
    flight_attrs_tmp['type'] = 'stop'
    return flight_attrs_tmp
    

def getDetailedFlightLegInfo(flight_leg):
    try:
        flight_attrs_tmp = dict()
        
        flight_attrs_tmp['type'] = 'leg'

        flight_time = re.split('\n', flight_leg.find_element_by_class_name("flight-leg__flight-time").text)
        flight_date = re.split('\n', flight_leg.find_element_by_class_name("flight-leg__flight-date").text)

        flight_date_time_from = getDetailedFlightLegDatetimeInfo(flight_time, flight_date, 'from')

        flight_date_time_to = getDetailedFlightLegDatetimeInfo(flight_time, flight_date, 'to')

        flight_attrs_tmp['date_time'] = {'from' : flight_date_time_from,
                                         'to' : flight_date_time_to}

        flight_attrs_tmp['airport'] = getDetailedFlightLegAirportInfo(flight_leg)

        flight_airline = flight_leg.find_element_by_class_name("flight-leg__airline-logo")
        flight_attrs_tmp['airline'] = flight_airline.find_element_by_css_selector('img').get_attribute('alt')
        
        flight_number = flight_leg.find_element_by_class_name("flight-leg__flight-info-link")
        flight_attrs_tmp['flight_number'] = flight_number.find_element_by_css_selector("span").text[19:]
    
    except NoSuchElementException as exception:
        print("\x1b[31m\"Element not found (getDetailedFlightLegInfo): ", exception, "\"\x1b[0m")
    return flight_attrs_tmp


def getDetailedFlightLegAirportInfo(flight_leg):
    flight_airport = flight_leg.find_element_by_class_name("flight-leg__flight-city")
    flight_airport_from = flight_airport.find_elements_by_tag_name('div')[0].text
    flight_airport_to = flight_airport.find_elements_by_tag_name('div')[1].text
    flight_airport_info = {'from' : 
                                   {'name' : flight_airport_from[4:],
                                    'iata' : flight_airport_from[:4]},
                                   'to' : 
                                   {'name' : flight_airport_to[4:],
                                    'iata' : flight_airport_to[:4]}}
    return flight_airport_info


def getDetailedFlightLegDatetimeInfo(flight_time, flight_date, point_key):
    point_index = -1
    if point_key == 'from':
        point_index = 0
    elif point_key == 'to':
        point_index = 1
    else:
        raise(ValueError)
    
    flight_time_point = re.split(':', flight_time[point_index])
    flight_date_point = re.split(' ', flight_date[point_index])

    #ATTENTION: создаю багу того что год в дате проставляется как год даты начала поездки
    flight_date_time_point = datetime.datetime(2019, #######hardcode######################################
                                               translateDate(flight_date_point[1][:-1]),
                                               int(flight_date_point[0]),
                                               int(flight_time_point[0]),
                                               int(flight_time_point[1])) 
    return flight_date_time_point


In [6]:
driver = webdriver.Firefox()
driver.get('https://www.aviasales.ru/search/MOW1307STO1')

In [4]:
def crawl_by_list(destinations, dates, driver): # dates - что то в духе ['1304','1404', ...]
    collection = getCollection()
    query = 'MOW'
    for date in dates:
        for destination in destinations:
            query = 'MOW' + date + destination + '1'
            goToPage(query, driver)
            processPage(driver, collection)

In [10]:
crawl_by_list(city_list, time_list_april[21:], driver)

**********************************************************************************************************************
$$$$$   page loaded for 15 seconds   $$$$$
MOW → ALC 

crawl_date_time:  2019-02-19 09:26:59.392634
price:  9262
	total_time:  {'minutes': '10', 'hours': '5'}
	date_time:  {'to': datetime.datetime(2019, 4, 22, 15, 10), 'from': datetime.datetime(2019, 4, 22, 11, 0)}
	city:  {'to': 'Аликанте', 'from': 'Москва'}


crawl_date_time:  2019-02-19 09:27:01.287049
price:  10811
	total_time:  {'minutes': '20', 'hours': '5'}
	date_time:  {'to': datetime.datetime(2019, 4, 22, 14, 10), 'from': datetime.datetime(2019, 4, 22, 9, 50)}
	city:  {'to': 'Аликанте', 'from': 'Москва'}


**********************************************************************************************************************
$$$$$   page loaded for 18 seconds   $$$$$
MOW → AMS 

crawl_date_time:  2019-02-19 09:27:24.018570
price:  7372
	total_time:  {'minutes': '10', 'hours': '22'}
	date_time:  {'to': datetime.d

crawl_date_time:  2019-02-19 09:28:57.367291
price:  10551
"Element not found (getDetailedFlightLegInfo):  Message: Unable to locate element: .flight-leg__flight-info-link
 "
"Element not found (getDetailedFlightLegInfo):  Message: Unable to locate element: .flight-leg__flight-info-link
 "
"Element not found (getDetailedFlightLegInfo):  Message: Unable to locate element: .flight-leg__flight-info-link
 "
	total_time:  {'minutes': '55', 'hours': '27'}
	date_time:  {'to': datetime.datetime(2019, 4, 23, 23, 30), 'from': datetime.datetime(2019, 4, 22, 20, 35)}
	city:  {'to': 'Барселона', 'from': 'Москва'}


crawl_date_time:  2019-02-19 09:29:00.094160
price:  10828
	total_time:  {'minutes': 0, 'hours': '18'}
	date_time:  {'to': datetime.datetime(2019, 4, 23, 0, 20), 'from': datetime.datetime(2019, 4, 22, 7, 20)}
	city:  {'to': 'Барселона', 'from': 'Москва'}


crawl_date_time:  2019-02-19 09:29:02.552932
price:  10853
"Element not found (getDetailedFlightLegInfo):  Message: Unable to locate 

**********************************************************************************************************************
$$$$$   page loaded for 23 seconds   $$$$$
MOW → BER 

crawl_date_time:  2019-02-19 09:30:32.939272
price:  4712
	total_time:  {'minutes': '40', 'hours': '2'}
	date_time:  {'to': datetime.datetime(2019, 4, 22, 15, 5), 'from': datetime.datetime(2019, 4, 22, 13, 25)}
	city:  {'to': 'Берлин', 'from': 'Москва'}


crawl_date_time:  2019-02-19 09:30:36.193708
price:  7386
	total_time:  {'minutes': '55', 'hours': '2'}
	date_time:  {'to': datetime.datetime(2019, 4, 22, 14, 35), 'from': datetime.datetime(2019, 4, 22, 12, 40)}
	city:  {'to': 'Берлин', 'from': 'Москва'}


crawl_date_time:  2019-02-19 09:30:38.583119
price:  8383
"Element not found (getDetailedFlightLegInfo):  Message: Unable to locate element: .flight-leg__flight-info-link
 "
	total_time:  {'minutes': '50', 'hours': '2'}
	date_time:  {'to': datetime.datetime(2019, 4, 22, 12, 45), 'from': datetime.datetime(2019, 4

crawl_date_time:  2019-02-19 09:33:04.848332
price:  11924
	total_time:  {'minutes': 0, 'hours': '39'}
	date_time:  {'to': datetime.datetime(2019, 4, 23, 17, 10), 'from': datetime.datetime(2019, 4, 22, 3, 10)}
	city:  {'to': 'Бремен', 'from': 'Москва'}


**********************************************************************************************************************
$$$$$   page loaded for 20 seconds   $$$$$
MOW → BRQ 

**********************************************************************************************************************
$$$$$   page loaded for 22 seconds   $$$$$
MOW → OST 

crawl_date_time:  2019-02-19 09:33:52.616529
price:  21849
"Element not found (getDetailedFlightLegInfo):  Message: Unable to locate element: .flight-leg__flight-info-link
 "
"Element not found (getDetailedFlightLegInfo):  Message: Unable to locate element: .flight-leg__flight-info-link
 "
	total_time:  {'minutes': '20', 'hours': '26'}
	date_time:  {'to': datetime.datetime(2019, 4, 23, 12, 20),

	total_time:  {'minutes': '45', 'hours': '22'}
	date_time:  {'to': datetime.datetime(2019, 4, 23, 18, 20), 'from': datetime.datetime(2019, 4, 22, 20, 35)}
	city:  {'to': 'Брюссель', 'from': 'Москва'}


crawl_date_time:  2019-02-19 09:34:59.749996
price:  7393
	total_time:  {'minutes': '20', 'hours': '23'}
	date_time:  {'to': datetime.datetime(2019, 4, 23, 10, 30), 'from': datetime.datetime(2019, 4, 22, 12, 10)}
	city:  {'to': 'Брюссель', 'from': 'Москва'}


crawl_date_time:  2019-02-19 09:35:02.322675
price:  7393
	total_time:  {'minutes': '10', 'hours': '24'}
	date_time:  {'to': datetime.datetime(2019, 4, 23, 14, 0), 'from': datetime.datetime(2019, 4, 22, 14, 50)}
	city:  {'to': 'Брюссель', 'from': 'Москва'}


crawl_date_time:  2019-02-19 09:35:05.306063
price:  7393
	total_time:  {'minutes': '10', 'hours': '26'}
	date_time:  {'to': datetime.datetime(2019, 4, 23, 21, 45), 'from': datetime.datetime(2019, 4, 22, 20, 35)}
	city:  {'to': 'Брюссель', 'from': 'Москва'}


*******************

...

crawl_date_time:  2019-02-19 10:33:58.562746
price:  6772
	total_time:  {'minutes': '25', 'hours': '19'}
	date_time:  {'to': datetime.datetime(2019, 4, 23, 15, 5), 'from': datetime.datetime(2019, 4, 22, 20, 40)}
	city:  {'to': 'Стокгольм', 'from': 'Москва'}


crawl_date_time:  2019-02-19 10:34:01.948416
price:  6772
	total_time:  {'minutes': '55', 'hours': '21'}
	date_time:  {'to': datetime.datetime(2019, 4, 23, 12, 20), 'from': datetime.datetime(2019, 4, 22, 15, 25)}
	city:  {'to': 'Стокгольм', 'from': 'Москва'}


crawl_date_time:  2019-02-19 10:34:06.104663
price:  6772
	total_time:  {'minutes': '5', 'hours': '23'}
	date_time:  {'to': datetime.datetime(2019, 4, 23, 18, 45), 'from': datetime.datetime(2019, 4, 22, 20, 40)}
	city:  {'to': 'Стокгольм', 'from': 'Москва'}


crawl_date_time:  2019-02-19 10:34:10.488602
price:  6772
	total_time:  {'minutes': '40', 'hours': '24'}
	date_time:  {'to': datetime.datetime(2019, 4, 23, 15, 5), 'from': datetime.datetime(2019, 4, 22, 15, 25)}
	city: 

crawl_date_time:  2019-02-19 10:37:16.907485
price:  27708
	total_time:  {'minutes': '35', 'hours': '24'}
	date_time:  {'to': datetime.datetime(2019, 4, 23, 7, 50), 'from': datetime.datetime(2019, 4, 22, 14, 15)}
	city:  {'to': 'Монреаль', 'from': 'Москва'}


crawl_date_time:  2019-02-19 10:37:19.560947
price:  27708
	total_time:  {'minutes': '49', 'hours': '24'}
	date_time:  {'to': datetime.datetime(2019, 4, 23, 8, 4), 'from': datetime.datetime(2019, 4, 22, 14, 15)}
	city:  {'to': 'Монреаль', 'from': 'Москва'}


crawl_date_time:  2019-02-19 10:37:22.313533
price:  27972
	total_time:  {'minutes': '54', 'hours': '17'}
	date_time:  {'to': datetime.datetime(2019, 4, 22, 20, 14), 'from': datetime.datetime(2019, 4, 22, 9, 20)}
	city:  {'to': 'Монреаль', 'from': 'Москва'}


crawl_date_time:  2019-02-19 10:37:24.707437
price:  27972
"Element not found (getDetailedFlightLegInfo):  Message: Unable to locate element: .flight-leg__flight-info-link
 "
	total_time:  {'minutes': '4', 'hours': '19'}


MOW → ZRH 

crawl_date_time:  2019-02-19 10:40:31.252873
price:  7378
	total_time:  {'minutes': '40', 'hours': '13'}
	date_time:  {'to': datetime.datetime(2019, 4, 23, 9, 15), 'from': datetime.datetime(2019, 4, 22, 20, 35)}
	city:  {'to': 'Цюрих', 'from': 'Москва'}


crawl_date_time:  2019-02-19 10:40:33.664263
price:  7378
	total_time:  {'minutes': '10', 'hours': '18'}
	date_time:  {'to': datetime.datetime(2019, 4, 23, 13, 45), 'from': datetime.datetime(2019, 4, 22, 20, 35)}
	city:  {'to': 'Цюрих', 'from': 'Москва'}


crawl_date_time:  2019-02-19 10:40:36.710903
price:  7378
	total_time:  {'minutes': '25', 'hours': '19'}
	date_time:  {'to': datetime.datetime(2019, 4, 23, 9, 15), 'from': datetime.datetime(2019, 4, 22, 14, 50)}
	city:  {'to': 'Цюрих', 'from': 'Москва'}


crawl_date_time:  2019-02-19 10:40:39.057015
price:  7378
	total_time:  {'minutes': '5', 'hours': '22'}
	date_time:  {'to': datetime.datetime(2019, 4, 23, 9, 15), 'from': datetime.datetime(2019, 4, 22, 12, 10)}
	city:  

"Element not found (getDetailedFlightLegInfo):  Message: Unable to locate element: .flight-leg__flight-info-link
 "
	total_time:  {'minutes': '40', 'hours': '20'}
	date_time:  {'to': datetime.datetime(2019, 4, 24, 8, 25), 'from': datetime.datetime(2019, 4, 23, 12, 45)}
	city:  {'to': 'Аликанте', 'from': 'Москва'}


crawl_date_time:  2019-02-19 10:43:11.438358
price:  11157
"Element not found (getDetailedFlightLegInfo):  Message: Unable to locate element: .flight-leg__flight-info-link
 "
	total_time:  {'minutes': 0, 'hours': '28'}
	date_time:  {'to': datetime.datetime(2019, 4, 24, 8, 40), 'from': datetime.datetime(2019, 4, 23, 5, 40)}
	city:  {'to': 'Аликанте', 'from': 'Москва'}


crawl_date_time:  2019-02-19 10:43:14.024368
price:  11157
"Element not found (getDetailedFlightLegInfo):  Message: Unable to locate element: .flight-leg__flight-info-link
 "
	total_time:  {'minutes': '40', 'hours': '28'}
	date_time:  {'to': datetime.datetime(2019, 4, 24, 20, 40), 'from': datetime.datetime(201

MaxRetryError: HTTPConnectionPool(host='127.0.0.1', port=38877): Max retries exceeded with url: /session/b0174b23-e32b-4100-8539-b37143a21880/elements (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x7fa5b4376240>: Failed to establish a new connection: [Errno 111] Connection refused',))

In [5]:
#март
time_list_march = ['0103','0203','0303','0403','0503','0603','0703','0803','0903','1003',
                   '1103','1203','1303','1403','1503','1603','1703','1803','1903','2003',
                   '2103','2203','2303','2403','2503','2603','2703','2803','2903','3003','3103']

#апрель
time_list_april = ['0104','0204','0304','0404','0504','0604','0704','0804','0904','1004',
                   '1104','1204','1304','1404','1504','1604','1704','1804','1904','2004',
                   '2104','2204','2304','2404','2504','2604','2704','2804','2904','3004']

city_list = {'BER':'Берлин', 
             'HAM':'Гамбург', 
             'BRE':'Бремен', 
             'HAJ':'Ганновер', 
             'LEJ':'Лейпциг', 
             'DUS':'Дюссельдорф', 
             'CGN':'Кёльн', 
             'NUE':'Нюрнберг', 
             'MUC':'Мюнхен', 
             'PRG':'Прага', 
             'BRQ':'Брно', 
             'KLV':'Карловы Вары', 
             'VIE':'Вена', 
             'SZG':'Зальцбург', 
             'INN':'Инсбрук', 
             'BUD':'Будапешт', 
             'BRU':'Брюссель', 
             'ANR':'Антверпен', 
             'OST':'Брюгге', 
             'NCE':'Ницца', 
             'LYS':'Лион', 
             'PAR':'Париж', 
             'MPL':'Монпелье', 
             'SXB':'Страсбург', 
             'GVA':'Женева', 
             'ZRH':'Цюрих', 
             'BRN':'Берн',
             'MIL':'Милан',
             'ROM':'Рим',
             'VCE':'Венеция',
             'VRN':'Верона',
             'NAP':'Неаполь',
             'TRN':'Турин',
             'FLR':'Флоренция',
             'PMO':'Палермо',
             'BLQ':'Болонья',
             'PEG':'Перуджа',
             'AGP':'Малага',
             'BCN':'Барселона',
             'PMI':'Майорка',
             'MAD':'Мадрид',
             'ALC':'Аликанте',
             'SVQ':'Севилья',
             'VLC':'Валенсия',
             'QCN':'Таррагона',
             'IBZ':'Ибица',
             'LON':'Лондон',
             'MAN':'Манчестер',
             'EDI':'Эдинбург',
             'GLA':'Глазго',
             'CBG':'Кембридж',
             'BFS':'Белфаст',
             'DUB':'Дублин',
             'ORC':'Корк',
             'AMS':'Амстердам',
             'RTM':'Роттердам',
             'EIN':'Эйндховен',
             'LCA':'Ларнака',
             'ECN':'Никосия',
             'PFO':'Пафос',
             'STO':'Стокгольм',
             'GOT':'Гётерберг'
             }

city_list = OrderedDict(sorted(city_list.items(), key=lambda kv: kv[1]))

In [417]:
def printBeautifulTicket(post):
    print('_id: ', post['_id'])
    print('crawl_date_time: ', post['crawl_date_time'])
    print('flights: ')
    for flight in post['flights']:
        print('\tcity: ')
        print('\t\tfrom\t: ', flight['city']['from'])
        print('\t\tto\t: ', flight['city']['to'])
        print('\tdate_time: ')
        print('\t\tfrom\t: ', flight['date_time']['from'])
        print('\t\tto\t: ', flight['date_time']['to'])
        print('\tdetailes: ')
        for period in flight['details']:
            if period['type'] == 'leg':
                print('\t\tairline\t: ', period['airline'])
                print('\t\tairport: ')
                print('\t\t\tfrom: ')
                print('\t\t\t\tiata\t: ', period['airport']['from']['iata'])
                print('\t\t\t\tname\t: ', period['airport']['from']['name'])
                print('\t\t\tto: ')
                print('\t\t\t\tiata\t: ', period['airport']['to']['iata'])
                print('\t\t\t\tname\t: ', period['airport']['to']['name'])
                print('\t\tdate_time: ')
                print('\t\t\tfrom\t: ', period['date_time']['from'])
                print('\t\t\tto\t: ', period['date_time']['to'])
                print('\t\tflight_number\t: ', period['flight_number'])
                print('\t\ttype\t: ', period['type'])
            if period['type'] == 'stop':
                print('\t\tplace\t: ', period['place'])
                print('\t\ttime: ')
                print('\t\t\thours\t: ', period['time']['hours'])
                print('\t\t\tminutes\t: '), period['time']['minutes']
                
    print('\n\n')

In [419]:
for post in collection.find({'price':61765}):
    printBeautifulTicket(post)

_id:  5c60f9a9bee9fd0d3239381f
crawl_date_time:  2019-02-11 07:27:14.324000
flights: 
	city: 
		from	:  Москва
		to	:  Антананариву
	date_time: 
		from	:  2019-07-13 09:40:00
		to	:  2019-07-14 15:35:00
	detailes: 
		airline	:  Qatar Airways
		airport: 
			from: 
				iata	:  DME 
				name	:  Домодедово
			to: 
				iata	:  DOH 
				name	:  Доха
		date_time: 
			from	:  2019-07-13 09:40:00
			to	:  2019-07-13 14:50:00
		flight_number	:  QR-224
		type	:  leg
		place	:  Пересадка в Дохе
		time: 
			hours	:  2
			minutes	: 
		airline	:  Qatar Airways
		airport: 
			from: 
				iata	:  DOH 
				name	:  Доха
			to: 
				iata	:  NBO 
				name	:  Джомо Кеньятта
		date_time: 
			from	:  2019-07-13 17:40:00
			to	:  2019-07-13 23:35:00
		flight_number	:  QR-1341
		type	:  leg
		place	:  Пересадка в Найроби
		time: 
			hours	:  12
			minutes	: 
		airline	:  Kenya Airways
		airport: 
			from: 
				iata	:  NBO 
				name	:  Джомо Кеньятта
			to: 
				iata	:  TNR 
				name	:  Антананариву
		date_time: 

In [420]:
for post in collection.find({'price':61765}):
    pprint(post)

{'_id': ObjectId('5c60f9a9bee9fd0d3239381f'),
 'crawl_date_time': datetime.datetime(2019, 2, 11, 7, 27, 14, 324000),
 'flights': [{'city': {'from': 'Москва', 'to': 'Антананариву'},
              'date_time': {'from': datetime.datetime(2019, 7, 13, 9, 40),
                            'to': datetime.datetime(2019, 7, 14, 15, 35)},
              'details': [{'airline': 'Qatar Airways',
                           'airport': {'from': {'iata': 'DME ',
                                                'name': 'Домодедово'},
                                       'to': {'iata': 'DOH ', 'name': 'Доха'}},
                           'date_time': {'from': datetime.datetime(2019, 7, 13, 9, 40),
                                         'to': datetime.datetime(2019, 7, 13, 14, 50)},
                           'flight_number': 'QR-224',
                           'type': 'leg'},
                          {'place': 'Пересадка в Дохе',
                           'time': {'hours': 2, 'minutes': 50},
      

             {'city': {'from': 'Антананариву', 'to': 'Москва'},
              'date_time': {'from': datetime.datetime(2019, 7, 20, 3, 25),
                            'to': datetime.datetime(2019, 7, 21, 8, 0)},
              'details': [{'airline': 'Kenya Airways',
                           'airport': {'from': {'iata': 'TNR ',
                                                'name': 'Антананариву'},
                                       'to': {'iata': 'NBO ',
                                              'name': 'Джомо Кеньятта'}},
                           'date_time': {'from': datetime.datetime(2019, 7, 20, 3, 25),
                                         'to': datetime.datetime(2019, 7, 20, 6, 50)},
                           'flight_number': 'KQ-260',
                           'type': 'leg'},
                          {'place': 'Пересадка в Найроби',
                           'time': {'hours': 3, 'minutes': 10},
                           'type': 'stop'},
                     